In [1]:
import io
import dropbox

# 1. Init dropbox object
In order to keep the **dropbox token** as secret we are storing it in a file that is in the `.gitignore`. This way it won't be shared

In [2]:
with open("dropbox.secret", "r") as file:
    token = file.read()
    
DBX = dropbox.Dropbox(token)

# 2. Writting files
## 2.1. Write a text file

In [3]:
txt = "Hello World"

with io.BytesIO(txt.encode()) as stream:
    stream.seek(0)

    # Write a text file
    DBX.files_upload(stream.read(), "/test.txt")

## 2.2. Write a json

In [4]:
import json

data = {"a": 1, "b": "hey"}

with io.StringIO() as stream:
    json.dump(data, stream, indent=4) # Ident param is optional

    stream.seek(0)

    DBX.files_upload(stream.read().encode(), "/test.json")

## 2.3. Write a yaml

In [5]:
import yaml

data = {"a": 1, "b": "hey"}

with io.StringIO() as stream:
    yaml.dump(data, stream, default_flow_style=False)

    stream.seek(0)

    DBX.files_upload(stream.read().encode(), "/test.yaml")

## 2.4. Write an Excel

In [6]:
import pandas as pd

df = pd.DataFrame([range(5), list("ABCDE")])

with io.BytesIO() as stream:

    with pd.ExcelWriter(stream) as writer:
        df.to_excel(writer)
        writer.save()
    
    stream.seek(0)

    DBX.files_upload(stream.getvalue(), "/test.xlsx")

## 2.5. Write a csv

In [7]:
df = pd.DataFrame([range(5), list("ABCDE")])

data = df.to_csv(index=False) # The index parameter is optional

with io.BytesIO(data.encode()) as stream:
    stream.seek(0)

    DBX.files_upload(stream.read(), "/test.csv")

# 3. Reading files
## 3.1. Read a text file

In [8]:
_, res = DBX.files_download("/test.txt")

with io.BytesIO(res.content) as stream:
    txt = stream.read().decode()
    
print(txt)

Hello World


## 3.2. Read a json

In [9]:
_, res = DBX.files_download("/test.json")

with io.BytesIO(res.content) as stream:
    data = json.load(stream)
    
data

{'a': 1, 'b': 'hey'}

## 3.3. Read a yaml

In [10]:
_, res = DBX.files_download("/test.yaml")

with io.BytesIO(res.content) as stream:
    data = yaml.safe_load(stream)

data

{'a': 1, 'b': 'hey'}

## 3.4. Read an Excel

In [11]:
_, res = DBX.files_download("/test.xlsx")

with io.BytesIO(res.content) as stream:
    df = pd.read_excel(stream, index_col=0)

df

,0,1,2,3,4
0,0,1,2,3,4
1,A,B,C,D,E


## 3.5. Read a csv

In [12]:
_, res = DBX.files_download("/test.csv")

with io.BytesIO(res.content) as stream:
    df = pd.read_csv(stream, index_col=0)
    
df

,1,2,3,4
0,,,,
0,1,2,3,4
A,B,C,D,E


# 4. Deleting files
We will delete all files created in this example

In [13]:
for extension in ["txt", "json", "yaml", "xlsx", "csv"]:
    
    filename = "/test." + extension
    
    try:
        DBX.files_delete(filename)
        print(f"File '{filename}' deleted")

    except dropbox.exceptions.ApiError:
        print(f"File '{filename}' was not found")

File '/test.txt' deleted
File '/test.json' deleted
File '/test.yaml' deleted
File '/test.xlsx' deleted
File '/test.csv' deleted
